In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import spacy
from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from config import config

# Connect to the database server
connection = None
try:
    params = config()
    connection = psycopg2.connect(**params)
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

# load model to NER
nlp = spacy.load("en_core_sci_sm")


In [2]:
query_schema = """set search_path to mimiciii;"""

# Counts the number of patients with a diagnosis 
query="""select icd9_code, short_title, long_title, count(distinct subject_id) as number_patients 
from diagnoses_icd join d_icd_diagnoses using(icd9_code)
group by icd9_code, short_title, long_title order by number_patients desc;"""
df = pd.read_sql_query(query_schema + query, connection)
df.head(20)
# print(len(df.index))

,icd9_code,short_title,long_title,number_patients
0,4019,Hypertension NOS,Unspecified essential hypertension,17613
1,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,10775
2,42731,Atrial fibrillation,Atrial fibrillation,10271
3,4280,CHF NOS,"Congestive heart failure, unspecified",9843
4,5849,Acute kidney failure NOS,"Acute kidney failure, unspecified",7687
5,2724,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia,7465
6,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...,7370
7,51881,Acute respiratry failure,Acute respiratory failure,6719
8,5990,Urin tract infection NOS,"Urinary tract infection, site not specified",5779
9,V053,Need prphyl vc vrl hepat,Need for prophylactic vaccination and inoculat...,5776


In [3]:
# Search for patients with Acute respiratory failure
query="""select distinct subject_id, icd9_code from diagnoses_icd where icd9_code='51881' order by subject_id;"""
pat_arf = pd.read_sql_query(query_schema + query, connection)
pat_arf.head()

,subject_id,icd9_code
0,91,51881
1,101,51881
2,111,51881
3,112,51881
4,141,51881


In [12]:
# Search for total of patients in mimic iii
query = """select count(DISTINCT subject_id) as total from patients"""
df_notes = pd.read_sql_query(query_schema + query, connection)
df_notes.head(24)

,total
0,46520


In [11]:
# Search for total of patients with noteevents
query = """select count(DISTINCT subject_id) as total from noteevents"""
df_notes = pd.read_sql_query(query_schema + query, connection)
df_notes.head(24)

,total
0,46146


In [20]:
# Search for average of noteevents by patient
query = """select avg(NE.total) as media from (select subject_id, count(DISTINCT row_id) as total from noteevents group by subject_id) as NE"""
df_notes = pd.read_sql_query(query_schema + query, connection)
df_notes.head(24)
# print(len(df_notes.index))

,media
0,45.143241


In [9]:
# Search for free text notes of patients with acute respiratory failure
query = """select subject_id, ne.hadm_id, di.icd9_code, chartdate, category, description, text 
from noteevents as ne join diagnoses_icd as di using(subject_id) where icd9_code='51881' 
order by subject_id limit 40"""
df_notes = pd.read_sql_query(query_schema + query, connection)
df_notes.head(24)

,subject_id,hadm_id,icd9_code,chartdate,category,description,text
0,91,121205.0,51881,2177-05-05,Discharge summary,Report,Admission Date: [**2177-4-23**] Discharge...
1,91,121205.0,51881,2177-05-10,Discharge summary,Addendum,"Name: [**Known lastname **], [**Known firstna..."
2,91,121205.0,51881,2177-05-07,ECG,Report,Sinus tachycardia. Low voltage throughout. Del...
3,91,121205.0,51881,2177-04-26,ECG,Report,Sinus tachycardia\nGeneralized low QRS voltage...
4,91,121205.0,51881,2177-04-22,ECG,Report,Sinus tachycardia\nLeft axis deviation\nPoor R...
5,91,NaN,51881,2177-03-27,Radiology,CT PELVIS W/CONTRAST,[**2177-3-27**] 3:09 PM\n CT ABDOMEN W/CONTRAS...
6,91,121205.0,51881,2177-05-07,Radiology,INITAL 3RD ORDER ABD/PEL/LOWER EXT A-GRAM,[**2177-5-7**] 2:03 PM\n MESENTERIC ...
7,91,121205.0,51881,2177-04-29,Radiology,UD GUID FOR NEEDLE PLACMENT,[**2177-4-29**] 12:24 PM\n PICC LINE PLACMENT ...
8,91,121205.0,51881,2177-04-23,Radiology,CT PELVIS W/CONTRAST,[**2177-4-23**] 1:51 AM\n CT ABDOMEN W/CONTRAS...
9,91,121205.0,51881,2177-04-27,Nursing/other,Report,SICU ADMISSION NOTE 230-730\n PT IS A 81YO FE...


In [5]:
# Top 10 most common lab procedures in patients with acute respiratory failure
query = """select count(procedures.icd9_code) as occurrences, procedures.icd9_code, short_title, long_title
from (select procedures_icd.icd9_code, short_title, long_title, subject_id
from procedures_icd join d_icd_procedures using(icd9_code)) as procedures join (select distinct subject_id, icd9_code from diagnoses_icd where icd9_code='51881') as patient using(subject_id)
group by procedures.icd9_code, short_title, long_title order by occurrences desc;"""
top_procedures = pd.read_sql_query(query_schema + query, connection)
top_procedures.head()

,occurrences,icd9_code,short_title,long_title
0,5724,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified"
1,5086,9604,Insert endotracheal tube,Insertion of endotracheal tube
2,3785,9671,Cont inv mec ven <96 hrs,Continuous invasive mechanical ventilation for...
3,3466,966,Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...
4,3347,9672,Cont inv mec ven 96+ hrs,Continuous invasive mechanical ventilation for...


In [5]:
# Professionals who write the most
query = """select label, cg.description as type_caregiver, count(cg.description) as number_professionals from caregivers as cg join 
(select subject_id, ne.cgid, ne.hadm_id, di.icd9_code, chartdate, category, ne.description, text from noteevents as ne 
join diagnoses_icd as di using(subject_id) where icd9_code='51881') as NE_ARF using(cgid)
group by label, type_caregiver order by number_professionals desc;
"""
count_caregivers = pd.read_sql_query(query_schema + query, connection)
count_caregivers.head()

,label,type_caregiver,number_professionals
0,RN,RN,226941
1,RRT,Respiratory,57707
2,Res,Resident/Fellow/PA/NP,36465
3,MD,Attending,29461
4,MDs,Resident/Fellow/PA/NP,19629


In [7]:
for text in df_notes['text']:
    print(text)

Admission Date:  [**2177-4-23**]     Discharge Date:


Service:

NOTE:  This is a preliminary Discharge Summary based on the
[**Hospital 228**] hospital course while she was on the Medicine
Service prior to being transferred to the Oncology Service.

HISTORY OF PRESENT ILLNESS:  Ms. [**Known firstname 26681**] [**Known lastname 17744**] is an
81-year-old female with a complicated past medical history
who presented with several months of fevers as well as rash
of approximately three weeks duration.

The patient's past medical history dates back to [**2172**], at
which time she had a medical workup for abdominal pain,
weight loss, and anorexia.  She had a CAT scan done on
[**2173-1-5**] that showed significant abdominal
adenopathy, extensive adenopathy displacing the bowel in the
periaortic, aorta cava, and mesenteric area.  She had a bone
marrow biopsy done on [**2173-1-13**] that showed no
evidence of malignancy.  In [**2173-2-4**], the patient then
had a retroperitoneal node biopsy do

In [10]:

def match_procedure(doc):
    for t in doc:
        for p in top_procedures['short_title']:
            if t.lower_ in p:
                return True
        return False

for text in df_notes['text']:
    doc = nlp(text)
    print(doc.ents)
    print('\n#######\n')
    match_procedure(doc)

displacy.render(doc, style='ent')
# displacy.render(next(doc.sents), style='dep', jupyter=True)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

(Admission, NOTE, hospital course, Medicine
Service, transferred, Oncology Service, HISTORY OF PRESENT ILLNESS, female, medical history
, months, fevers, rash, weeks, duration, patient's, medical history, medical workup, abdominal pain, anorexia, CAT scan, abdominal
adenopathy, adenopathy, bowel, aorta cava, mesenteric area, bone
marrow biopsy, [**2173, evidence, malignancy, patient, retroperitoneal, node biopsy, mini-laparotomy, granulomatous lymphadenitis, acid-fast bacillus
negative, GMS, negative, PAS, negative, patient, diagnosis, sarcoidosis, patient, low-grade, satiety, abdominal pain, abnormal xylose
test, [**2173**, working
, diagnosis, patient, sarcoidosis, blood, tested, negative, histoplasmosis, blastomycosis, negative, started, time, year, [**2175**, patient, diagnosed, Crohn's disease, clinical diagnosis, documented, colonoscopy, patient, Dr. [**First Name4, gastroenterologist, outside, facility, medications, treat, Crohn's disease, 6-MP, methotrexate, Remicade, Remicade,

PT ENTITY
CONT ENTITY
VSS ENTITY
LOPRESSOR IV GIVEN ENTITY
MARGINAL RESULTS ENTITY
MSO4 ENTITY
GIVEN ENTITY
GOOD ENTITY
RELIEF/RESULT-PT ON PROPOFOL ENTITY
GTT ENTITY
BUT SEEN TO BE GRIMACING AS IF ENTITY
PAIN ENTITY
LOW ENTITY
HCT-26 ENTITY
RECTAL ENTITY
BAG ENTITY
CHECK ENTITY
RETURNED ENTITY
STERILE H2O FLUSH ENTITY
PT ENTITY
VENTED ENTITY
TIME ENTITY
ON ENTITY
GTT ENTITY
RIGHT ENTITY
STABLE/UNCHANGED ENTITY
LEFT ENTITY
OOZE ENTITY
THOUGH ENTITY
Location ENTITY


In [4]:
if connection is not None:
    connection.close()
    print('Database connection closed.')

Database connection closed.
